<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Generative_AI_for_Personalized_Recipe_Generation_at_Safeway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To implement the most advanced approach for Generative AI for Personalized Recipe Generation at Safeway, the system can be built around a fine-tuned GPT/LLama/Claudo model that not only generates personalized recipes but also integrates with Safeway's product data, customer preferences, and dietary restrictions. This system can dynamically generate recipe ideas and suggest complementary products, ultimately enhancing cross-selling and upselling opportunities.

1. Data Collection and Preparation
Customer Purchase History: Collect customer purchase data (e.g., past purchases, product categories, favorite items) and store it in a database.
Product Data: Catalog all products with detailed metadata (e.g., ingredients, nutrition facts, dietary information, stock availability, prices).
Dietary Preferences and Restrictions: Allow customers to input dietary preferences (e.g., vegetarian, vegan, gluten-free) and restrictions (e.g., allergies to nuts, lactose intolerance).
Local Sales Data: Integrate current promotions and local sales data for recommending products that are on discount or in abundance.

In [ ]:
import pandas as pd

# Load product data
products = pd.read_csv('product_catalog.csv')

# Example product data structure
product_data = {
    'product_id': [1, 2, 3, 4],
    'name': ['Pasta', 'Olive Oil', 'Cheese', 'Tomatoes'],
    'category': ['Grain', 'Oil', 'Dairy', 'Vegetable'],
    'price': [2.99, 5.49, 3.99, 1.49],
    'dietary_tags': ['vegetarian', 'vegan', 'gluten-free', ''],
    'in_stock': [True, True, False, True]
}

# Customer dietary preferences and restrictions
customer_preferences = {
    'vegetarian': True,
    'gluten_free': False,
    'nut_allergy': True
}

# Sales data
local_sales = {
    'Pasta': 0.10,  # 10% discount
    'Tomatoes': 0.15  # 15% discount
}


In [ ]:
2. Fine-Tuning a GPT Model for Recipe Generation
Model Choice: Use an advanced generative model such as GPT-4 or a fine-tuned variant like T5 for text generation, fine-tuned on a recipe dataset.
Fine-Tuning: Fine-tune the GPT model on a dataset of recipes, incorporating ingredients, cooking steps, and metadata (dietary labels, meal types, etc.).
Data for Fine-Tuning: Datasets like RecipeNLG can be used to fine-tune the model to generate recipes dynamically.
Fine-Tuning Code Example (Using Hugging Face Transformers):

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# Load pretrained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Fine-tune on recipe dataset
train_data = tokenizer("data/recipe_dataset.txt", return_tensors='pt', padding=True, truncation=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data
)

# Start training
trainer.train()


3. Generating Personalized Recipes
The fine-tuned model will generate personalized recipes based on a customer's purchase history and preferences. The model should dynamically adjust the recipe depending on available products, dietary restrictions, and current sales. For instance, if the customer bought pasta, the system would suggest a pasta recipe while incorporating local sales (e.g., olive oil on discount).

Advanced Approach:

Product Matching: Match the customer’s past purchases with products in the inventory.
Dietary Consideration: Filter recipe ingredients based on customer preferences and dietary restrictions (e.g., vegetarian, gluten-free).
Sales Integration: Highlight ingredients that are currently on sale to increase cross-selling.
Code Example for Personalized Recipe Generation:

In [ ]:
def generate_personalized_recipe(purchase_history, dietary_preferences, product_data, local_sales):
    # Use GPT model to generate a recipe based on the customer's past purchases
    prompt = f"Generate a recipe for {', '.join(purchase_history)} considering the following dietary preferences: {', '.join([key for key, value in dietary_preferences.items() if value])}."

    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    generated_ids = model.generate(input_ids, max_length=150, num_return_sequences=1)

    recipe = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Suggest complementary ingredients and apply discounts
    suggested_products = []
    for product in product_data['name']:
        if product in local_sales:
            suggested_products.append(f"{product} (on sale: {local_sales[product] * 100}% off)")
        else:
            suggested_products.append(product)

    return recipe, suggested_products

# Example usage
purchase_history = ['Pasta', 'Tomatoes']
recipe, suggestions = generate_personalized_recipe(purchase_history, customer_preferences, product_data, local_sales)
print(f"Recipe: {recipe}")
print(f"Suggested products: {suggestions}")


4. Cross-Selling and Upselling
After generating the recipe, the system should recommend complementary products (e.g., olive oil, cheese) that the customer can easily add to their shopping cart.
By using the customer's purchase history and sales data, the system can highlight discounts to encourage cross-selling and upselling.
Advanced Features:

Dynamic Suggestions: As new products go on sale or items go out of stock, the system can update its recommendations in real-time.
Shopping Cart Integration: Allow customers to click and add all suggested ingredients to their cart directly from the recipe page.
Code Example for Adding to Cart:

In [ ]:
def suggest_complementary_products(recipe_ingredients, product_data):
    complementary_products = []
    for ingredient in recipe_ingredients:
        product_match = product_data[product_data['name'].str.contains(ingredient)]
        if not product_match.empty and product_match['in_stock'].values[0]:
            complementary_products.append({
                'name': product_match['name'].values[0],
                'price': product_match['price'].values[0],
                'on_sale': local_sales.get(product_match['name'].values[0], 0)
            })
    return complementary_products

# Example complementary products for the generated recipe
complementary_products = suggest_complementary_products(['Pasta', 'Olive Oil'], product_data)
print(complementary_products)


5. Real-Time Product Availability and Sales Integration
Integrate the system with Safeway’s inventory management to ensure that only available products are recommended.
If certain ingredients are unavailable or on sale, the system can adapt the recipe accordingly.
Code Example (Checking Real-Time Availability):

In [ ]:
def check_availability_and_sales(ingredient, product_data, local_sales):
    product_match = product_data[product_data['name'].str.contains(ingredient)]
    if not product_match.empty:
        product_info = product_match.iloc[0]
        availability = product_info['in_stock']
        sale = local_sales.get(product_info['name'], 0)
        return availability, sale
    return False, 0

# Example check for a specific product
availability, sale = check_availability_and_sales('Pasta', product_data, local_sales)
if availability:
    print(f"Pasta is available with {sale * 100}% off")


6. Deployment and Scaling
Deploy the system on cloud infrastructure such as AWS Lambda or Google Cloud Functions for scalable serverless architecture.
APIs can be created to allow integration with Safeway’s mobile app or website, where customers can receive personalized recipe suggestions in real-time.
API Endpoint Example Using Flask:

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/generate_recipe', methods=['POST'])
def generate_recipe():
    data = request.json
    purchase_history = data.get('purchase_history')
    recipe, suggestions = generate_personalized_recipe(purchase_history, customer_preferences, product_data, local_sales)
    return jsonify({'recipe': recipe, 'suggestions': suggestions})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


Advanced Features to Consider:
Recipe Customization: Allow customers to tweak recipes (e.g., change serving size or substitute ingredients based on availability).
Voice Integration: Implement voice commands for generating recipes using Amazon Alexa or Google Assistant.
Nutritional Information: Automatically calculate and display nutritional values for the generated recipes based on product data.

This advanced approach leverages a fine-tuned GPT model to generate personalized recipes based on customer preferences, past purchases, and local sales. It integrates real-time product availability and suggests complementary items to enhance cross-selling and upselling opportunities, ultimately creating a seamless shopping experience for Safeway customers.